In [1]:
!pwd
import sys
sys.path.append('/workspace')

/workspace/notebooks


In [4]:
from typing import List, Optional

from pandas import DataFrame
import pandas as pd

import json

from src.sources.data_source import DataSource

from src.mapping.pdfs.pdf_field_name_classifier import FieldNameClassifier
from src.mapping.pdfs.pdf_field_label_catalog import FieldLabelCatalog
from src.mapping.pdfs.pseudofield_generator import PseudofieldGenerator
from src.mapping.rows.row_mapping_configuration import RowMappingConfiguration
from src.mapping.values.value_matching_configuration import ValueMatchingConfiguration
from src.core.db.config import DatabaseEnum
from src.core.db.models.pdf_models import Fincen8300Rev4
from src.core.db.session import DBContext, DbQuery, AppSession
from src.mapping.columns.column_relation import ColumnRelation

In [15]:
from sqlalchemy.orm.query import Query

from src.core.db.models.main_models import Employee
with DBContext(DatabaseEnum.MAIN_INGESTION_DB) as main_db:
    results: Query = main_db.query(Employee)
    print(results.count())
    print(len(results.all()))




[03/04/2021 21:22:48] [    INFO] [sqlalchemy.engine.base.Engine] [_begin_impl():730] [PID:26 TID:140675330815808] -> BEGIN (implicit)
[03/04/2021 21:22:48] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:26 TID:140675330815808] -> SELECT count(*) AS count_1 
FROM (SELECT employee.id AS employee_id, employee.prefix_name AS employee_prefix_name, employee.first_name AS employee_first_name, employee.last_name AS employee_last_name, employee.middle_name AS employee_middle_name, employee.suffix_name AS employee_suffix_name, employee.date_of_birth AS employee_date_of_birth, employee.dod_id AS employee_dod_id, employee.ssn AS employee_ssn 
FROM employee) AS anon_1
[03/04/2021 21:22:48] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:26 TID:140675330815808] -> {}
[03/04/2021 21:22:48] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:26 TID:140675330815808] -> SELECT employee.id AS employee_id, employee.prefix_name AS e

0
0


In [9]:
db = DbQuery(DatabaseEnum.MAIN_INGESTION_DB)
result = db.execute("SELECT * from public.employee")
df = DataFrame(result.fetchall())
df.shape[0]

[06/04/2021 02:17:30] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:26 TID:139699430995776] -> SELECT * from public.employee
[06/04/2021 02:17:30] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:26 TID:139699430995776] -> {}


2005

In [50]:
from src.core.db.models.main_models import (
    ComplianceRunEvent,
    EmployeeToComplianceRunEvent,   
    Employee, DocumentType
)
fields = ['ssn', 'date_of_birth', 'first_name', 'last_name']
app_emp = AppSession(DatabaseEnum.MAIN_INGESTION_DB)
session_emp = app_emp.instance
query = (
        session_emp.query(Employee).with_entities(Employee.ssn, Employee.date_of_birth, 
                                              Employee.first_name, Employee.last_name)
        .statement
    )
df_emp = pd.read_sql(query, app_emp.engine)
app_emp.instance.close()
df_emp.head()

[06/04/2021 04:02:46] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:26 TID:139699430995776] -> select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
[06/04/2021 04:02:46] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:26 TID:139699430995776] -> {'name': 'SELECT employee.ssn, employee.date_of_birth, employee.first_name, employee.last_name \nFROM employee'}
[06/04/2021 04:02:46] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:26 TID:139699430995776] -> SELECT employee.ssn, employee.date_of_birth, employee.first_name, employee.last_name 
FROM employee
[06/04/2021 04:02:46] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:26 TID:139699430995776] -> {}


,ssn,date_of_birth,first_name,last_name


In [51]:
df_emp.shape

(0, 4)

In [36]:
print(DocumentType.name)

DocumentType.name


In [49]:
with DBContext(DatabaseEnum.MAIN_INGESTION_DB) as main_db:
    # Have to use "%" because there are hidden characters in the db load from csv
    doc_type = main_db.query(DocumentType).filter(DocumentType.name.like('%fincen%')).one_or_none()    
    print(doc_type.id)
     

[06/04/2021 03:11:21] [    INFO] [sqlalchemy.engine.base.Engine] [_begin_impl():730] [PID:26 TID:139699430995776] -> BEGIN (implicit)
[06/04/2021 03:11:21] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:26 TID:139699430995776] -> SELECT document_type.id AS document_type_id, document_type.name AS document_type_name 
FROM document_type 
WHERE document_type.name LIKE %(name_1)s
[06/04/2021 03:11:21] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:26 TID:139699430995776] -> {'name_1': '%fincen%'}
[06/04/2021 03:11:21] [    INFO] [sqlalchemy.engine.base.Engine] [_commit_impl():772] [PID:26 TID:139699430995776] -> COMMIT


99990e40-5ca6-437c-8d72-adb1a66d7f78


# Before Hard-Coding Column Mappings

Better to hard-code them because we know the table structure and don't want to have to recompute it every time.

In [7]:
class DynamicCompliance:
    def __init__(self):
        self.employee = self._get_employee_data_source()

        # If employee is None there is no employee table
        if self.employee:
            value_matching_config_json = self._load_config(
                "../config/mapping/levenshtein_default.json"
            )
            row_mapping_config_json = self._load_config(
                "../config/mapping/weighted_linear_default.json"
            )
            self.value_matching_config = ValueMatchingConfiguration(
                **value_matching_config_json
            )
            self.row_mapping_config = RowMappingConfiguration(**row_mapping_config_json)
            self.fincen_column_relations = self._get_fincen_column_relations()

    def _get_fincen_column_relations(self) -> List[ColumnRelation]:
        fincen = self._get_fincen_data_source()

        pseudofield_generator = PseudofieldGenerator(fincen)
        pseudofield_generator.generate()

        # noinspection PyTypeChecker
        self._create_column_relations_for(fincen, self.employee)
        # noinspection PyTypeChecker
        return fincen.column_relations

    @staticmethod
    def _get_fincen_data_source() -> DataSource:
        db = DbQuery(DatabaseEnum.PDF_INGESTION_DB)
        result = db.execute("SELECT * from public.fincen8300_rev4 limit 10")
        df = DataFrame(result.fetchall())
        df.columns = result.keys()
        fincen = DataSource(df)
        return fincen

    @staticmethod
    def _get_employee_data_source() -> Optional[DataSource]:
        db = DbQuery(DatabaseEnum.MAIN_INGESTION_DB)
        result = db.execute("SELECT * from public.employee")
        df = DataFrame(result.fetchall())
        if df.shape[0] == 0:
            employee = None
        else:
            df.columns = result.keys()
            employee = DataSource(df)
        return employee

    @staticmethod
    def _create_column_relations_for(source, target):
        """Create column relations from canonical column identifiers."""
        gold_id_info = FieldNameClassifier.get_id_info_from_df(target.get_data())
        data_id_info = FieldNameClassifier.get_id_info_from_df(source.get_data())
        for identifier in FieldLabelCatalog:
            if identifier in gold_id_info and identifier in data_id_info:
                g_id = gold_id_info[identifier]
                d_id = data_id_info[identifier]
                source.create_column_relation(d_id.field_name, g_id.field_name, target)
                print("New relation detected: %s" % str(source.column_relations[-1]))

    @staticmethod
    def _load_config(path) -> dict:
        """
        Read JSON from a filepath
        """
        with open(path, "r") as F:
            return json.load(F)

    def generate_structured_row_matches(self, source: DataSource) -> DataFrame:
        """Generate structured row matches."""
        rows = {
            "first_name": [],  # just for sanity check
            "last_name": [],  # just for sanity check
            "ingestion_event_id": [],
            "employee_id": [],
        }
        # noinspection PyUnresolvedReferences
        for relation in source.row_relations:
            source_index = relation.source_index
            target_index = relation.target_index
            # noinspection PyUnresolvedReferences
            source_row = source.get_data().iloc[source_index]
            # noinspection PyUnresolvedReferences
            target_row = self.employee.get_data().iloc[target_index]
            rows["employee_id"].append(target_row.id)
            rows["ingestion_event_id"].append(source_row.ingestion_event_id)
            rows["first_name"].append(source_row.first_name)
            rows["last_name"].append(source_row.last_name)
        return DataFrame(rows)

    @staticmethod
    def _get_pdf_document(ingestion_event_id: str) -> pd.DataFrame:
        app_pdf = AppSession(DatabaseEnum.PDF_INGESTION_DB)
        session_pdf = app_pdf.instance
        # only reasonable way to get into a dataframe
        query = (
            session_pdf.query(Fincen8300Rev4)
            .filter(Fincen8300Rev4.ingestion_event_id == ingestion_event_id)
            .statement
        )
        df = pd.read_sql(query, app_pdf.engine)
        app_pdf.instance.close()
        return df

    @staticmethod
    def get_ingestion_event_and_write_to_compliance(ingestion_event_id: str):
        with DBContext(DatabaseEnum.PDF_INGESTION_DB) as pdf_db:
            result = (
                pdf_db.query(IngestionEvent)
                .filter(IngestionEvent.id == ingestion_event_id)
                .one_or_none()
            )
            if result is None:
                return None
            with DBContext(DatabaseEnum.MAIN_INGESTION_DB) as main_db:
                main_db.add(
                    ComplianceRunEvent(
                        id=ingestion_event_id,
                        s3_bucket=result.s3_bucket,
                        s3_key=result.s3_key,
                        was_redacted=False,
                        status="ok",
                    )
                )
        return "done"

    def filter_and_retain(self, ingestion_event_id: str):

        if self.employee is None:
            return "No records in employee table"

        # first get ingestion event
        r = self.get_ingestion_event_and_write_to_compliance(ingestion_event_id)
        if r is None:
            return "ingestion_event_id not found"
        df = self._get_pdf_document(ingestion_event_id)
        f_vals = df.to_dict(orient="records")[0]  # assume one doc per ingestion_event
        num_document_matches = df.shape[0]
        fincen = DataSource(df)
        fincen.column_relations = self.fincen_column_relations
        fincen.map_rows_to(
            self.employee, self.value_matching_config, self.row_mapping_config
        )
        results_df = self.generate_structured_row_matches(fincen)
        num_records = results_df.shape[0]

        # Write to EmployeeToComplianceRunEvent
        if num_records > 0:
            row = results_df.iloc[0]
            with DBContext(DatabaseEnum.MAIN_INGESTION_DB) as main_db:
                main_db.add(
                    EmployeeToComplianceRunEvent(
                        employee_id=str(row.employee_id),
                        compliance_run_event_id=ingestion_event_id,
                    )
                )

            # Write to Fincen
            del f_vals["ingestion_event_id"]
            with DBContext(DatabaseEnum.MAIN_INGESTION_DB) as main_db:
                main_db.add(FincenMain(**f_vals))

        return (
            f"Num documents matched: {num_document_matches}, "
            f"Num employees matched: {num_records}"
        )


In [4]:
cm = ColumnMap()

[29/03/2021 00:46:56] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:215 TID:139832163526464] -> select version()
[29/03/2021 00:46:56] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:215 TID:139832163526464] -> {}
[29/03/2021 00:46:56] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:215 TID:139832163526464] -> select current_schema()
[29/03/2021 00:46:56] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:215 TID:139832163526464] -> {}
[29/03/2021 00:46:56] [    INFO] [sqlalchemy.engine.base.Engine] [_cursor_execute():1341] [PID:215 TID:139832163526464] -> SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
[29/03/2021 00:46:56] [    INFO] [sqlalchemy.engine.base.Engine] [_cursor_execute():1342] [PID:215 TID:139832163526464] -> {}
[29/03/2021 00:46:56] [    INFO] [sqlalchemy.engine.base.Engine] [_cursor_execute():1341] [PID:215 TID:139832163526464] -> SELECT CAST('test unicode 

New relation detected: tin -> pandas DataFrame (hash 477683055509494867): ssn (1.00 conf)
New relation detected: dob -> pandas DataFrame (hash 477683055509494867): date_of_birth (1.00 conf)
New relation detected: first_name -> pandas DataFrame (hash 477683055509494867): first_name (1.00 conf)
New relation detected: last_name -> pandas DataFrame (hash 477683055509494867): last_name (1.00 conf)


In [8]:
dc = DynamicCompliance()
for cr in dc.fincen_column_relations:
    print(cr)

[06/04/2021 02:07:15] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:26 TID:139699430995776] -> SELECT * from public.employee
[06/04/2021 02:07:15] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:26 TID:139699430995776] -> {}
[06/04/2021 02:07:15] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:26 TID:139699430995776] -> select version()
[06/04/2021 02:07:15] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:26 TID:139699430995776] -> {}
[06/04/2021 02:07:15] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:26 TID:139699430995776] -> select current_schema()
[06/04/2021 02:07:15] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:26 TID:139699430995776] -> {}
[06/04/2021 02:07:15] [    INFO] [sqlalchemy.engine.base.Engine] [_cursor_execute():1341] [PID:26 TID:139699430995776] -> SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
[

New relation detected: tin -> pandas DataFrame (hash 5727275029921275503): ssn (1.00 conf)
New relation detected: dob -> pandas DataFrame (hash 5727275029921275503): date_of_birth (1.00 conf)
New relation detected: first_name -> pandas DataFrame (hash 5727275029921275503): first_name (1.00 conf)
New relation detected: last_name -> pandas DataFrame (hash 5727275029921275503): last_name (1.00 conf)
tin -> pandas DataFrame (hash 5727275029921275503): ssn (1.00 conf)
dob -> pandas DataFrame (hash 5727275029921275503): date_of_birth (1.00 conf)
first_name -> pandas DataFrame (hash 5727275029921275503): first_name (1.00 conf)
last_name -> pandas DataFrame (hash 5727275029921275503): last_name (1.00 conf)


In [5]:
def filter_and_retain(ingestion_event_id: str):
    app_pdf = AppSession(DatabaseEnum.PDF_INGESTION_DB)
    session_pdf = app_pdf.instance
    # only reasonable way to get into a dataframe
    query = (
        session_pdf.query(Fincen8300Rev4)
        .filter(Fincen8300Rev4.ingestion_event_id == ingestion_event_id)
        .statement
    )
    df = pd.read_sql(query, app_pdf.engine)
    
    num_document_matches = df.shape[0]
    fincen = DataSource(df)
    fincen.column_relations = cm.fincen_column_relations
    fincen.map_rows_to(
        cm.employee, cm.value_matching_config, cm.row_mapping_config
    )
    print(fincen.row_relations)
    results_df = cm.generate_structured_row_matches(fincen)
    num_records = results_df.shape[0]
    with DBContext(DatabaseEnum.MAIN_INGESTION_DB) as main_db:
        for i in range(num_records):
            row = results_df.iloc[i]
            main_db.add(
                EmployeeToDocument(
                    employee_id=str(row.employee_id),
                    ingestion_event_id=str(row.ingestion_event_id),
                )
            )
    return f"Num documents matched: {num_document_matches}, " \
           f"Num employees matched: {num_records}"

In [6]:
# This person not in employee database
ingestion_event_id = "17d0e319-c247-432f-9d91-f4259430f25b"
m = filter_and_retain(ingestion_event_id)
print(m)

[29/03/2021 00:46:56] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:215 TID:139832163526464] -> select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
[29/03/2021 00:46:56] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:215 TID:139832163526464] -> {'name': 'SELECT fincen8300_rev4.id, fincen8300_rev4.amends_prior_report, fincen8300_rev4.suspicious_transaction, fincen8300_rev4.multiple_individuals, fincen8 ... (2168 characters truncated) ... ate_cash_received, fincen8300_rev4.date_of_signature_footer \nFROM fincen8300_rev4 \nWHERE fincen8300_rev4.ingestion_event_id = :ingestion_event_id_1'}
[29/03/2021 00:46:56] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:215 TID:139832163526464] -> SELECT fincen8300_rev4.id, fincen8300_rev4.amends_prior_report, fincen8300_rev4.suspicious_transaction, fincen8300_rev4.multiple_individuals

[]
Num documents matched: 1, Num employees matched: 0


In [7]:
# This person in employee database
ingestion_event_id = "58320e40-5ca6-437c-8d72-adb1a66d7f78"
m = filter_and_retain(ingestion_event_id)
print(m)

[29/03/2021 00:46:57] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:215 TID:139832163526464] -> select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
[29/03/2021 00:46:57] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:215 TID:139832163526464] -> {'name': 'SELECT fincen8300_rev4.id, fincen8300_rev4.amends_prior_report, fincen8300_rev4.suspicious_transaction, fincen8300_rev4.multiple_individuals, fincen8 ... (2168 characters truncated) ... ate_cash_received, fincen8300_rev4.date_of_signature_footer \nFROM fincen8300_rev4 \nWHERE fincen8300_rev4.ingestion_event_id = :ingestion_event_id_1'}
[29/03/2021 00:46:57] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:215 TID:139832163526464] -> SELECT fincen8300_rev4.id, fincen8300_rev4.amends_prior_report, fincen8300_rev4.suspicious_transaction, fincen8300_rev4.multiple_individuals

[RowRelation(target_data_source=pandas DataFrame (hash 477683055509494867), source_index=0, target_index=13, confidence=0.8838090114490482)]
Num documents matched: 1, Num employees matched: 1
